## Multiclass Venn-ABERS calibration  - Satellite Image dataset

This notebook illustrates the multi-class Venn-ABERS calibration as implemented in:

Manokhin, Valery. "Multi-class probabilistic classification using inductive and cross Venn–Abers predictors." In Conformal and Probabilistic Prediction and Applications, pp. 228-240. PMLR, 2017.

As an example we use the Satellite Image dataset described in more detail here: https://www.openml.org/search?type=data&sort=runs&id=182&status=active

## Import libraries

In [19]:
import pandas as pd
import numpy as np
from scipy.io import arff
from sklearn.metrics import log_loss, brier_score_loss, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from sklearn.calibration import CalibratedClassifierCV

from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

import warnings
warnings.filterwarnings('ignore')

from venn_abers import VennAbersCalibrator

## Load dataset

In [20]:
data = arff.loadarff('dataset_186_satimage.arff')
df = pd.DataFrame(data[0])
df['class'] = ([int(str(i)[2]) for i in df['class'].values])
df['class'] = df['class'].astype('int')-1

In [21]:
df.head()

Aattr     Battr     Cattr     Dattr     Eattr     Fattr    A1attr  \
0  0.117596  1.241362  1.184036  0.815302 -0.158561  1.256483  1.193546   
1 -1.205362 -1.249654 -0.077532  0.444886 -0.895959 -0.447579 -0.786760   
2  0.779075  0.148811  0.042617 -0.243030  0.800057  0.164136  0.053370   
3  1.146564  0.585831  0.342991  0.021553  0.947536  0.601074  0.353416   
4 -0.764376 -1.162250 -0.137607  0.180303 -0.969698 -1.146681 -0.126658   

     B2attr    C3attr    D4attr  ...   D22attr   E23attr   F24attr   A25attr  \
0  0.818486 -0.141965  0.879481  ...  0.807707 -0.069968  1.219160  1.250463   
1 -0.554203 -0.364672  0.092157  ... -0.192752 -0.736996 -0.969292 -0.844805   
2 -0.448612  0.154978 -0.345245  ... -0.877277  0.671174 -0.006373 -0.425752   
3  0.026550  1.788164  1.010702  ...  0.281150  1.412317  1.044084  0.532085   
4  0.184937 -0.735851 -1.132569  ... -0.192752 -0.885225 -1.231906 -0.784941   

    B26attr   C27attr   D28attr   E29attr   F30attr  class  
0  0.597678 -0.054291  1.233342  1.262255  0.603258      0  
1 -0.400030 -0.725852 -0.344432 -0.594534 -0.183967      4  
2 -0.662584  0.691889  0.356801 -0.175259 -0.236449      6  
3  0.282612  1.438068  1.058033  0.842981  0.130923      2  
4 -0.347519 -0.875088 -1.220973 -0.774223 -0.551339      4  

[5 rows x 37 columns]

In [22]:
df['class'].value_counts()

class
0    1531
6    1508
2    1356
4     707
1     703
3     625
Name: count, dtype: int64

There are five classed, with labels 0-6 (class label 5 is missing)

We split the data into a proper training set, calibration set and test set

In [23]:
df_train_cal, df_test = train_test_split(df, test_size = 2000, random_state = 42, shuffle = False)
df_proper_train, df_cal = train_test_split(df_train_cal, test_size = 2000, random_state = 42, shuffle = False)

In [24]:
X_train = df_train_cal.drop('class', axis=1)
y_train = df_train_cal['class']

X_proper_train = df_proper_train.drop('class', axis=1)
y_proper_train = df_proper_train['class']

X_cal = df_cal.drop('class', axis=1)
y_cal = df_cal['class']

X_test = df_test.drop('class', axis=1)
y_test = df_test['class']

In [25]:
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)

In [26]:
def brier_loss_calc(y_true, prob):
    return ((y_true - prob)**2).mean()

## Classifier comparison

We compare 5 underlying classifiers and two calibration methods (sigmoid and isotonic), both in their inductive and cross-validated format

In [27]:
clfs = {}
clfs['Naive Bayes'] = GaussianNB()
clfs['SVM'] = SVC(probability=True)
clfs['RF'] = RandomForestClassifier()
clfs['XGB'] = AdaBoostClassifier()
clfs['Logistic'] = LogisticRegression(max_iter=10000)
clfs['Neural Network'] =  MLPClassifier(max_iter=10000)


def run_multiclass_comparison(clf_name, clf):

    print(clf_name + ':')
    log_loss_list = []
    brier_loss_list = []
    acc_list = []
    ece_list = []
    
    print('base')
    clf.fit(X_train, y_train)
    p_pred = clf.predict_proba(X_test)
    y_pred = clf.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred))
    
    print('sigmoid')
    clf.fit(X_proper_train, y_proper_train)
    cal_sigm = CalibratedClassifierCV(clf, method='sigmoid', cv='prefit')
    cal_sigm.fit(X_cal, y_cal)
    p_pred = cal_sigm.predict_proba(X_test)
    y_pred = cal_sigm.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred))
    
    print('isotonic')
    cal_iso = CalibratedClassifierCV(clf, method='isotonic', cv='prefit')
    cal_iso.fit(X_cal, y_cal)
    p_pred = cal_iso.predict_proba(X_test)
    y_pred = cal_iso.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred))
    
    print('sigmoid_cv')
    cal_sigm_cv = CalibratedClassifierCV(clf, method='sigmoid', cv=5)
    cal_sigm_cv.fit(X_train, y_train)
    p_pred = cal_sigm_cv.predict_proba(X_test)
    y_pred = cal_sigm_cv.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred))
    
    print('isotonic_cv')
    cal_iso_cv = CalibratedClassifierCV(clf, method='isotonic', cv=5)
    cal_iso_cv.fit(X_train, y_train)
    p_pred = cal_iso_cv.predict_proba(X_test)
    y_pred = cal_iso_cv.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred))
    
    print('ivap')
    va = VennAbersCalibrator(clf, inductive=True, cal_size=0.2, random_state=42)
    va.fit(np.asarray(X_train), np.asarray(y_train))
    p_pred_va = va.predict_proba(np.array(X_test))
    y_pred = va.predict(np.array(X_test), one_hot=False)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred_va))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred_va))
    
    print('cvap')
    va_cv = VennAbersCalibrator(clf, inductive=False, n_splits=5)
    va_cv.fit(np.asarray(X_train), np.asarray(y_train))
    p_pred_cv = va_cv.predict_proba(np.asarray(X_test))
    y_pred = va_cv.predict(np.array(X_test), one_hot=False)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred_cv))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred_cv))

    print('')
    
    df_ll = pd.DataFrame(columns=['Classifier', 'Uncalibrated', 'Platt', 'Isotonic', 'Platt-CV', 'Isotonic-CV', 'IVAP', 'CVAP'])
    df_ll.loc[0] =  [clf_name] + log_loss_list
    df_bl = pd.DataFrame(columns=['Classifier', 'Uncalibrated', 'Platt', 'Isotonic', 'Platt-CV', 'Isotonic-CV', 'IVAP', 'CVAP'])
    df_bl.loc[0] =  [clf_name] + brier_loss_list
    df_acc = pd.DataFrame(columns=['Classifier', 'Uncalibrated', 'Platt', 'Isotonic', 'Platt-CV', 'Isotonic-CV', 'IVAP', 'CVAP'])
    df_acc.loc[0] =  [clf_name] + acc_list

    return df_bl, df_ll, df_acc

We compare accuracy, Brier and log loss

In [28]:
results_brier = pd.DataFrame()
results_log = pd.DataFrame()
results_acc = pd.DataFrame()

for clf_name in clfs:
    scratch_b, scratch_l, scratch_acc = run_multiclass_comparison(clf_name, clfs[clf_name])
    results_brier = pd.concat((results_brier, scratch_b), ignore_index=True)
    results_log = pd.concat((results_log, scratch_l), ignore_index=True)
    results_acc = pd.concat((results_acc, scratch_acc), ignore_index=True)
    

Naive Bayes:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap

SVM:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap

RF:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap

XGB:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap

Logistic:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap

Neural Network:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap



Calibrating can result in a slight improvement in accuracy

In [29]:
results_acc.set_index('Classifier', inplace=True)
results_acc.round(3)

Uncalibrated  Platt  Isotonic  Platt-CV  Isotonic-CV   IVAP  \
Classifier                                                                    
Naive Bayes            0.796  0.792     0.804     0.795        0.806  0.814   
SVM                    0.894  0.880     0.880     0.892        0.890  0.899   
RF                     0.902  0.890     0.890     0.905        0.902  0.897   
XGB                    0.750  0.518     0.576     0.740        0.770  0.885   
Logistic               0.862  0.802     0.822     0.790        0.814  0.872   
Neural Network         0.902  0.886     0.888     0.903        0.902  0.900   

                 CVAP  
Classifier             
Naive Bayes     0.814  
SVM             0.892  
RF              0.900  
XGB             0.884  
Logistic        0.870  
Neural Network  0.902

It generally improves Brier and log loss metrics

In [30]:
results_brier.set_index('Classifier', inplace=True)
results_brier.round(4)

Uncalibrated   Platt  Isotonic  Platt-CV  Isotonic-CV    IVAP  \
Classifier                                                                      
Naive Bayes           0.0657  0.0571    0.0480    0.0556       0.0469  0.0439   
SVM                   0.0257  0.0350    0.0309    0.0304       0.0274  0.0249   
RF                    0.0241  0.0254    0.0253    0.0233       0.0228  0.0246   
XGB                   0.0994  0.1025    0.0958    0.0744       0.0613  0.0282   
Logistic              0.0323  0.0529    0.0481    0.0515       0.0455  0.0298   
Neural Network        0.0270  0.0324    0.0269    0.0243       0.0222  0.0264   

                  CVAP  
Classifier              
Naive Bayes     0.0444  
SVM             0.0270  
RF              0.0248  
XGB             0.0280  
Logistic        0.0312  
Neural Network  0.0247

In [31]:
results_log.set_index('Classifier', inplace=True)
results_log.round(4)

Uncalibrated   Platt  Isotonic  Platt-CV  Isotonic-CV    IVAP  \
Classifier                                                                      
Naive Bayes           4.0378  0.7704    0.6101    0.7532       0.5960  0.4894   
SVM                   0.2806  0.4341    0.4587    0.3678       0.3335  0.2815   
RF                    0.2692  0.3005    0.3837    0.2721       0.2722  0.2854   
XGB                   1.1760  1.2685    1.2437    0.9034       0.7280  0.3278   
Logistic              0.3564  0.6104    0.6166    0.5951       0.5857  0.3310   
Neural Network        0.4727  0.4427    0.4013    0.3213       0.2407  0.3314   

                  CVAP  
Classifier              
Naive Bayes     0.5135  
SVM             0.3329  
RF              0.2957  
XGB             0.3403  
Logistic        0.3703  
Neural Network  0.3030

**Mean accuracy**

In [32]:
results_acc.mean()

Uncalibrated    0.850833
Platt           0.794500
Isotonic        0.809917
Platt-CV        0.837333
Isotonic-CV     0.847167
IVAP            0.877750
CVAP            0.877083
dtype: float64

**Average Brier and log loss ranking**

In [33]:
results_brier.rank(axis=1).mean()

Uncalibrated    4.500000
Platt           6.833333
Isotonic        5.166667
Platt-CV        4.000000
Isotonic-CV     2.666667
IVAP            2.166667
CVAP            2.666667
dtype: float64

In [34]:
results_log.rank(axis=1).mean()

Uncalibrated    3.833333
Platt           6.166667
Isotonic        6.000000
Platt-CV        4.000000
Isotonic-CV     3.000000
IVAP            2.166667
CVAP            2.833333
dtype: float64

**Mean Brier and log loss scores**

In [35]:
results_brier.mean()

Uncalibrated    0.045700
Platt           0.050871
Isotonic        0.045829
Platt-CV        0.043268
Isotonic-CV     0.037654
IVAP            0.029632
CVAP            0.030014
dtype: float64

In [36]:
results_log.mean()

Uncalibrated    1.098787
Platt           0.637776
Isotonic        0.619005
Platt-CV        0.535484
Isotonic-CV     0.459335
IVAP            0.341061
CVAP            0.359266
dtype: float64

Overall, in this example, IVAP and CVAP calibrators achieve highest overall accuracy and lowest Brier and log loss errors